# BERT

http://nlp.seas.harvard.edu/2018/04/03/attention.html

Encoder only transofrmer


1. Pretraining (Masked language modelling, next sentence prediction)
2. Finetuning (not sure how to do with shakespear)


Notes
- Senetence = arbitary span of contiguous text
- Sequence = input token sequence to bert, which may have multiple sentences
- Uses WordPiece embedings w 30k token vocab
- THe first ttoken of a sequeunece is a [CLS] = class token
- Sentences differentiaed by:
    - seperated in sequences by [SEP]
    - learned sequence embeddedding token
- Embedding = wordpiece token + segment token + position embedding. 

In [2]:

import torch

# Load dataset
with open("data/shakespear.txt") as fn:
    text = fn.read()

# Get unique lsit
chars = sorted(list(set(text)))

vocab_size = len(chars)
print(chars)
print(len(chars))

# Tokenize. Chars -> intergers/index
s2i = {ch: i for i, ch in enumerate(chars)}
i2s = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [s2i[ch] for ch in x]
decode = lambda x: [i2s[i] for i in x]

print(encode("hello"))
print(decode(encode("hello")))



['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65
[46, 43, 50, 50, 53]
['h', 'e', 'l', 'l', 'o']


In [3]:
# Tokenize whole training set
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

# Train/val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

torch.Size([1115393])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [4]:
context_length = 8
print(train_data[: context_length + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [ ]:
batch_size = 8
def get_batch():
    